## Bot Coronavirus-MG (@coronavirus_mg)

In [ ]:
import pandas as pd
import numpy as np
from datetime import date
import datetime
import wget
import os
import time

### Data e dia da semana

In [ ]:
data_de_hoje = str(date.today())
data_de_hoje

In [ ]:
data_de_hoje_texto_padrao_brasil = date.today().strftime('%d/%m/%Y')
print(data_de_hoje_texto_padrao_brasil)

In [ ]:
dia_semana = pd.Timestamp(data_de_hoje)
dia_semana_nome = str(dia_semana.day_name())
dia_semana_numero = str(dia_semana.day)

dia_semana_nome = dia_semana_nome.replace('Saturday', 'este sábado').replace('Sunday', 'este domingo').replace('Monday', 'esta segunda').replace('Tuesday', 'esta terça').replace('Wednesday', 'esta quarta').replace('Thursday', 'esta quinta').replace('Friday', 'esta sexta')

DIA_DA_SEMANA = dia_semana_nome + " (" + dia_semana_numero + ")"
print('DIA_DA_SEMANA:',DIA_DA_SEMANA)


### Seleção de conteúdo
Baixar o arquivo .XLSX Painel no site https://coronavirus.saude.mg.gov.br/painel

In [ ]:
boletim_atualizado = False
boletim_segue_para_analise = False
contador_boletim = 1

while boletim_segue_para_analise == False:
    
    #baixar o boletim:
    try: 
        wget.download('http://sescloud.saude.mg.gov.br/index.php/s/ZEzzC8jFpobXGjM/download?path=%2FPAINEL_COVID&files=XLSX_Painel.xlsx', 
                  'boletim.xlsx')
    except: 
        print("Arquivo não localizado. Verificar se não houve mudança de link")

    
    #ler o arquivo:
    abas = pd.ExcelFile('boletim.xlsx')
    confirmados = pd.read_excel(abas, 'CONFIRMADOS')
    obitos = pd.read_excel(abas, 'OBITOS')
    recuperados = pd.read_excel(abas, 'RECUPERADOS')
    
    #ver a ultima data de atualização das abas fundamentais
    #checar se o arquivo está atualizado com os dados de hoje:
    data_boletim_ok = None
    if datetime.datetime.date(confirmados['DATA'].max()) == datetime.date.today() or datetime.datetime.date(obitos['DATA'].max()) == datetime.date.today(): 
        data_boletim_ok = True
    else: 
        data_boletim_ok = False
    
    #Se estiver atualizada, quebra o while e segue o baile
    if data_boletim_ok == True:
        print('\nArquivo do boletim está atualizado. Pode seguir para a análise.\n')
        boletim_atualizado = True 
        boletim_segue_para_analise = True
    
    #Se não estiver atualizado, apaga o arquivo e refaz o processo 10 vezes dando um intervalo de 20 minutos entre as tentativas
    else: 
        if contador_boletim < 10:
            os.remove('boletim.xlsx')
            print(f'\nTentativa {contador_boletim} fracassada. Tentar novamente em 20 minutos.\n') 
            contador_boletim += 1 
            time.sleep(1200)
    #Se não estiver atualizado e falhar as 10 vezes, avisa que não atualizou e segue
    #Na hora de tuitar, se o arquivo do boletim não estiver atualizado, vai tuitar uma mensagem de erro
        else: 
            print('\nO arquivo não foi atualizado, mas a vida precisa seguir adiante\n')
            boletim_atualizado = False
            boletim_segue_para_analise = True

Antes de mais nada, limpar os códigos dos municípios porque eles chegam sem padrão (float, int, str). 
Datas já foram carregadas como Datetime e não será necessário fazer essa conversão

In [ ]:
confirmados['CodigoIBGE'] = confirmados['CodigoIBGE'].values.astype(int).astype(str)
obitos['CodigoIBGE'] = obitos['CodigoIBGE'].values.astype(int).astype(str)
recuperados['CodigoIBGE'] = recuperados['CodigoIBGE'].values.astype(int).astype(str)

### Casos Confirmados

1 - Criar uma tabela cruzada de casos confirmados por data, equivalente aos casos das últimas 24h <br>
2 - Criar uma coluna com a soma acumulada<br>
3 - Criar uma coluna de média móvel<br>

In [ ]:
#1
tabela_confirmados = confirmados.groupby(by=['DATA'], as_index=False)['NUM_CASOS'].agg(['sum'])
tabela_confirmados.rename(columns = {'sum' : 'casos_dia'}, inplace = True)

#2
tabela_confirmados['soma_acumulada'] = tabela_confirmados['casos_dia'].cumsum()

#3
tabela_confirmados['media_movel'] = tabela_confirmados['casos_dia'].rolling(7).mean().round(1)

tabela_confirmados.tail(7)

Extrair os valores do boletim diário e guardá-los em variáveis.<br>
casos_24h = casos confirmados a mais no boletim de hoje<br>
casos_total = total de casos confirmados até o momento<br>
casos_media = média móvel de casos confirmados nos últimos sete dias<br>
casos_media_anterior = média móvel de casos confirmados há duas semanas, para comparação da evolução<br>
casos_var = a variação percentual da média móvel nas últimas duas semanas<br>
casos_var_status = status da variação (em alta, estável ou em queda)

In [ ]:
casos_24h = tabela_confirmados.iloc[-1,0]
casos_total = tabela_confirmados.iloc[-1,1]
casos_media = tabela_confirmados.iloc[-1,2]
casos_media_anterior = tabela_confirmados.iloc[-15,2]
casos_var = ((casos_media - casos_media_anterior) / casos_media_anterior) * 100

casos_var_status = 0
if casos_var >= 15:
    casos_var_status = 'com tendência recente de alta'
elif casos_var <= -15:
    casos_var_status = 'com tendência recente de queda'
else: 
    casos_var_status = 'com tendência recente de estabilidade'

Padronizar as variáveis em texto no formato padrão (separador de milhar e decimal, percentual, etc).
Guardá-las em novas variáveis (CAIXA ALTA)) que serão utilizadas mais à frente nos templates de texto

In [ ]:
CASOS_24H = str('{0:,}'.format(casos_24h).replace(',','.'))
print('CASOS_24H:',CASOS_24H)

CASOS_TOTAL = str('{0:,}'.format(casos_total).replace(',','.'))
print('CASOS_TOTAL:',CASOS_TOTAL)

CASOS_MEDIA = str('{0:,}'.format((casos_media.round()).astype(int)).replace(',','.'))
print('CASOS_MEDIA:',CASOS_MEDIA)

CASOS_MEDIA_ANTERIOR = str('{0:,}'.format((casos_media_anterior.round()).astype(int)).replace(',','.'))
print('CASOS_MEDIA_ANTERIOR:',CASOS_MEDIA_ANTERIOR)

CASOS_VAR = casos_var.round(1).astype(str).replace('.',',')+'%'
print('CASOS_VAR:',CASOS_VAR)

CASOS_VAR_STATUS = casos_var_status
print('CASOS_VAR_STATUS:',CASOS_VAR_STATUS)

#### Criar o gráfico da média móvel de casos

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('seaborn')

In [ ]:
cor_casos = 'x'

if casos_var >= 15:
    cor_casos = 'red'
elif casos_var <= -15:
    cor_casos = 'darkgreen'
else: 
    cor_casos = 'chocolate'

In [ ]:
fig, ax = plt.subplots()

ax.set_title("CASOS CONFIRMADOS DE COVID-19 EM MINAS GERAIS\n(média móvel de sete dias, por data de confimação)")

ax.plot(tabela_confirmados.index,
        tabela_confirmados['media_movel'], 
        color='#41869f', 
        alpha=1)

ax.bar(tabela_confirmados.index,
       tabela_confirmados['casos_dia'],
       color='#41869f', 
       alpha=0.3)

ax.set_ylim(0, tabela_confirmados['casos_dia'].max())
ax.set_xlim(left=tabela_confirmados.index.min())

ax.set_ylabel("Novos casos")
ax.set_xlabel("\nFonte: SES-MG/Prefeituras  •  Análise: Cristiano Martins/@coronavirus_mg", color='grey', size=10)

ax.annotate('      '+CASOS_MEDIA, 
            xy = (pd.Timestamp(date.today()), casos_media+300),
            size = 16,
            weight="bold")
 
ax.annotate('        '+CASOS_VAR, 
            xy = (pd.Timestamp(date.today()), casos_media-800),
            color = cor_casos,
            size = 13)

ax.axhline(y = casos_media, color = 'grey', linestyle = 'dotted')
ax.axhline(y = 0, color = 'grey', linestyle = '-')

#ax.text(0.5, -0.15, 'Fonte: Prefeituras/SES-MG. Análise: Cristiano Martins / @coronavirus_mg', color='grey', size=11, horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)

fig.set_size_inches([16/2,9/2])

fig.savefig('grafico_media_casos.png', dpi = 300)

### Óbitos Confirmados

1 - Criar uma tabela cruzada de óbitos confirmados por data, equivalente aos óbitos das últimas 24h <br>
2 - Criar uma coluna com a soma acumulada <br>
3 - Criar uma coluna de média móvel <br>

In [ ]:
#1
tabela_obitos = obitos.groupby(by=['DATA'], as_index=False)['NUM_OBITOS'].agg(['sum'])
tabela_obitos.rename(columns = {'sum' : 'obitos_dia'}, inplace = True)

#2
tabela_obitos['soma_acumulada_obitos'] = tabela_obitos['obitos_dia'].cumsum()

#3
tabela_obitos['media_movel_obitos'] = tabela_obitos['obitos_dia'].rolling(7).mean().round(1)

tabela_obitos.tail(7)

Extrair os valores do boletim diário e guardá-los em variáveis.<br>
obitos_24h = obitos confirmados a mais no boletim de hoje<br>
obitos_total = total de obitos confirmados até o momento<br>
obitos_media = média móvel de obitos confirmados nos últimos sete dias<br>
obitos_media_anterior = média móvel de obitos confirmados há duas semanas, para comparação da evolução<br>
obitos_var = a variação percentual da média móvel dos obitos nas últimas duas semanas<br>
obitos_var_status = status da variação dos obitos (em alta, estável ou em queda)

In [ ]:
obitos_24h = tabela_obitos.iloc[-1,0]
obitos_total = tabela_obitos.iloc[-1,1]
obitos_media = tabela_obitos.iloc[-1,2]
obitos_media_anterior = tabela_obitos.iloc[-15,2]
obitos_var = ((obitos_media - obitos_media_anterior) / obitos_media_anterior) * 100

obitos_var_status = 0
if obitos_var >= 15:
    obitos_var_status = 'com tendência recente de alta'
elif obitos_var <= -15:
    obitos_var_status = 'com tendência recente de queda'
else: 
    obitos_var_status = 'com tendência recente de estabilidade'

Padronizar as variáveis no formato separado por milhar e decimal. Guardá-las em novas variáveis em caixa alta que serão utilizadas mais à frente nos templates de texto

In [ ]:
OBITOS_24H = str('{0:,}'.format(obitos_24h).replace(',','.'))
print('OBITOS_24H:',OBITOS_24H)

OBITOS_TOTAL = str('{0:,}'.format(obitos_total).replace(',','.'))
print('OBITOS_TOTAL:',OBITOS_TOTAL)

OBITOS_MEDIA = str('{0:,}'.format((obitos_media.round()).astype(int)).replace(',','.'))
print('OBITOS_MEDIA:',OBITOS_MEDIA)

OBITOS_MEDIA_ANTERIOR = str('{0:,}'.format((obitos_media_anterior.round()).astype(int)).replace(',','.'))
print('OBITOS_MEDIA_ANTERIOR:',OBITOS_MEDIA_ANTERIOR)

OBITOS_VAR = obitos_var.round(1).astype(str).replace('.',',')+'%'
print('OBITOS_VAR:',OBITOS_VAR)

OBITOS_VAR_STATUS = obitos_var_status
print('OBITOS_VAR_STATUS:',OBITOS_VAR_STATUS)

#### Criar o gráfico da média móvel de óbitos

In [ ]:
cor_obitos = 'z'

if obitos_var >= 15: 
    cor_obitos = 'red'
elif obitos_var <= -15: 
    cor_obitos = 'darkgreen'
else: 
    cor_obitos = 'chocolate'

In [ ]:
fig, ax = plt.subplots()

ax.set_title("MORTES CONFIRMADAS POR COVID-19 EM MINAS GERAIS\n(média móvel de sete dias, por data de confimação)")

ax.plot(tabela_obitos.index,
        tabela_obitos['media_movel_obitos'], 
        color='#825218', 
        alpha=1)

ax.bar(tabela_obitos.index,
       tabela_obitos['obitos_dia'],
       color='#825218', 
       alpha=.3)

ax.set_ylim(0, tabela_obitos['obitos_dia'].max())
ax.set_xlim(left=tabela_obitos.index.min())


ax.set_ylabel("Novas mortes")
ax.set_xlabel("\nFonte: SES-MG/Prefeituras  •  Análise: Cristiano Martins/@coronavirus_mg", color='grey', size=10)

ax.annotate('       '+OBITOS_MEDIA, 
            xy = (pd.Timestamp(date.today()), obitos_media + 5),
            size = 16,
            weight ='bold')

ax.annotate('       '+OBITOS_VAR, 
            xy = (pd.Timestamp(date.today()), obitos_media - 30),#-1300),
            size = 13, 
            color = cor_obitos)

ax.axhline(y = obitos_media, color = 'grey', linestyle = 'dotted')
ax.axhline(y = 0, color = 'grey', linestyle = '-')

#ax.text(0.5, -0.15, 'Fonte: SES-MG. Análise: Cristiano Martins / @coronavirus_mg', color='grey', size=11, horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)

fig.set_size_inches([16/2,9/2])

fig.savefig('grafico_media_obitos.png', dpi = 300)

### Casos Recuperados, casos ativos e letalidade
### OBS: Diferentemente dos casos e óbitos, a aba de casos recuperados informa sempre o total acumulado até o determinado dia.

### Criar tabela agrupada por data

In [ ]:
tabela_recuperados = recuperados.groupby(by=['DATA'], as_index=False)['NUM_RECUPERADOS'].agg(['sum'])
tabela_recuperados.rename(columns = {'sum' : 'recuperados_total'}, inplace = True)

tabela_recuperados.tail(7)

### recuperados_total = total acumulado de casos recuperados até a data
### recuperados_24h = total acumulado de casos recuperados a mais neste boletim

In [ ]:
recuperados_total = tabela_recuperados.iloc[-1,0]


### formatar para o texto e salvar na variável em caixa alta

In [ ]:
RECUPERADOS_TOTAL = str('{0:,}'.format(recuperados_total).replace(',','.'))
print('RECUPERADOS_TOTAL:',RECUPERADOS_TOTAL)

### Calcular a letalidade, isto é, a taxa proporcional de óbitos entre os casos confirmados

In [ ]:
letalidade = ((obitos_total * 100)/casos_total).round(1)
LETALIDADE = str('{0:,}'.format(letalidade).replace('.',',')) + '%'
print('LETALIDADE:',LETALIDADE)

### Casos ativos
##### criar série temporal de casos ativos a partir dos dados previamente tratados (total de casos menos óbitos e recuperados)

In [ ]:
#join das colunas já existentes pela data
tabela_ativos = tabela_confirmados.merge(tabela_obitos, how='inner', on='DATA')
tabela_ativos = tabela_ativos.merge(tabela_recuperados, how='inner', on='DATA')

#excluir colunas que não interessam aqui
tabela_ativos.drop(labels=['casos_dia','media_movel','obitos_dia','media_movel_obitos'], axis=1, inplace=True)

#calcular nova coluna 'ativos' a partir da subtratação do total
tabela_ativos['ativos'] = tabela_ativos['soma_acumulada'] - tabela_ativos['soma_acumulada_obitos'] - tabela_ativos['recuperados_total']

tabela_ativos.tail(7)

In [ ]:
ativos_atual = tabela_ativos.iloc[-1,-1]

In [ ]:
ATIVOS_ATUAL = str('{0:,}'.format(ativos_atual).replace(',','.'))
print('ATIVOS_ATUAL:',ATIVOS_ATUAL)

#### Gráfico de casos ativos

In [ ]:
fig, ax = plt.subplots()

ax.set_title("CASOS ATIVOS DE COVID-19 EM MINAS GERAIS\n(internados ou em acompanhamento)")

ax.plot(tabela_ativos.index,
        tabela_ativos['ativos'], 
        color='olive', 
        alpha=1)

ax.fill_between(tabela_ativos.index,
       tabela_ativos['ativos'],
       color='olive', 
       alpha=.3)

ax.set_ylim(0, tabela_ativos['ativos'].max()+40000)
ax.set_ylabel("Casos ativos")
ax.set_xlabel("\nFonte: SES-MG/Prefeituras  •  Análise: Cristiano Martins/@coronavirus_mg", color='grey', size=10)

ax.annotate('      '+ATIVOS_ATUAL, 
            xy = (pd.Timestamp(date.today()), ativos_atual-2000),
            size = 16,
            weight ='bold')

#ax.annotate('        '+OBITOS_VAR, 
#            xy = (pd.Timestamp(date.today()), obitos_media - 30),#-1300),
#            size = 13, 
#            color = cor_obitos)

ax.axhline(y = ativos_atual, color = 'grey', linestyle = 'dotted')
ax.axhline(y = 0, color = 'grey', linestyle = '-')

#ax.text(0.5, -0.15, 'Fonte: SES-MG. Análise: Cristiano Martins / @coronavirus_mg', color='grey', size=11, horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)

fig.set_size_inches([16/2,9/2])

fig.savefig('grafico_ativos.png', dpi = 300)

## Dados do Sivep de Internados
#### Baixar os dados do sistema sivep-gripe para analisar internações e letalidade entre os pacientes que desenvolveram SRAG por Covid-19

In [ ]:
try: 
    url2 = 'http://sescloud.saude.mg.gov.br/index.php/s/ZEzzC8jFpobXGjM/download?path=%2FSISTEMAS&files=XLSX_Sistemas.xlsx' 
    arquivo2 = 'sivep_mg.xlsx'
    wget.download(url2, arquivo2)
except:
    print("Arquivo não encontrado. Favor verificar se não houve troca de link.")

In [ ]:
sivep_mg = pd.read_excel('sivep_mg.xlsx')

#### 1) Filtrar apenas casos internados de srag confirmada por covid

In [ ]:
internados = sivep_mg[sivep_mg['INTERNACAO'] == 'SIM'].copy(deep=True)

In [ ]:
internados_total = internados.shape[0]
INTERNADOS_TOTAL = str('{0:,}'.format(internados_total).replace(',','.'))
print('INTERNADOS_TOTAL:',INTERNADOS_TOTAL)

##### Verificar a proporção de óbitos entre os internados

In [ ]:
tabela_internados_obitos = internados.groupby(by=['EVOLUCAO'])['EVOLUCAO'].agg(['count'])
tabela_internados_obitos

In [ ]:
internados_obitos = tabela_internados_obitos.iloc[1,0]
INTERNADOS_OBITOS = str('{0:,}'.format(internados_obitos).replace(',','.'))
print('INTERNADOS_OBITOS:',INTERNADOS_OBITOS)

In [ ]:
letalidade_internados = (internados_obitos*100)/internados_total
LETALIDADE_INTERNADOS = letalidade_internados.round(1).astype(str).replace('.',',')+'%'
print('LETALIDADE_INTERNADOS:',LETALIDADE_INTERNADOS)

#### 2) Filtrar apenas casos internados de srag confirmada por covid em **UTIs**

In [ ]:
internados_UTI = sivep_mg[sivep_mg['UTI'] == 'SIM']

In [ ]:
internados_UTI_total = internados_UTI.shape[0]
INTERNADOS_UTI_TOTAL = str('{0:,}'.format(internados_UTI_total).replace(',','.'))
print('INTERNADOS_UTI_TOTAL:',INTERNADOS_UTI_TOTAL)

##### Verificar a proporção de óbitos entre os internados em UTIs

In [ ]:
tabela_internados_UTI_obitos = internados_UTI.groupby(by=['EVOLUCAO'])['EVOLUCAO'].agg(['count'])
tabela_internados_UTI_obitos

In [ ]:
internados_UTI_obitos = tabela_internados_UTI_obitos.iloc[1,0]
INTERNADOS_UTI_OBITOS = str('{0:,}'.format(internados_UTI_obitos).replace(',','.'))
print('INTERNADOS_UTI_OBITOS:',INTERNADOS_UTI_OBITOS)

In [ ]:
letalidade_internados_UTI = (internados_UTI_obitos*100)/internados_UTI_total
LETALIDADE_INTERNADOS_UTI = letalidade_internados_UTI.round(1).astype(str).replace('.',',')+'%'
print('LETALIDADE_INTERNADOS_UTI:',LETALIDADE_INTERNADOS_UTI)

#### Gráfico da série temporal de internações

In [ ]:
internados['data_not'] = pd.to_datetime(internados['DATA_NOTIFICACAO'], format= "%d/%m/%Y")
internados['data_evol'] = pd.to_datetime(internados['DATA_EVOLUCAO'], format= "%d/%m/%Y")
internados['data_sint'] = pd.to_datetime(internados['DATA_1_SINTOMA'], format= "%d/%m/%Y")

In [ ]:
internados_datas = internados[(internados['data_sint'] >= '2020-03-08') & (internados['data_sint'] <= data_de_hoje)]

In [ ]:
#1
tabela_internados_datas = internados_datas.groupby(by=['data_sint'], as_index=False)['data_sint'].agg(['count']).reset_index()

tabela_internados_datas.rename(columns = {'count' : 'internados_dia'}, inplace = True)

#2
tabela_internados_datas['soma_acumulada_internados'] = tabela_internados_datas['internados_dia'].cumsum()

#3
tabela_internados_datas['media_movel_internados'] = tabela_internados_datas['internados_dia'].rolling(7).mean().round(1)

In [ ]:
tabela_internados_datas.tail()

In [ ]:
internados_data_recente = tabela_internados_datas.iloc[-1,0]
internados_data_recente

In [ ]:
import datetime
data_tres_semanas_atras = internados_data_recente - datetime.timedelta(days=21)
data_tres_semanas_atras

#### Gráficos da média móvel de internações

In [ ]:
tabela_incompleta_tres_semanas = tabela_internados_datas.copy(deep=True)

In [ ]:
tabela_incompleta_tres_semanas.iloc[-21:,-1] = None
tabela_incompleta_tres_semanas.iloc[-21:,1] = None

In [ ]:
tabela_incompleta_tres_semanas.tail(22)

In [ ]:
internados_media21 = tabela_incompleta_tres_semanas.iloc[-22,-1]
internados_media21

In [ ]:
INTERNADOS_MEDIA21 = str('{0:,}'.format((internados_media21.round()).astype(int)).replace(',','.'))
print('INTERNADOS_MEDIA21:',INTERNADOS_MEDIA21)

In [ ]:
fig, ax = plt.subplots()

#cinza por baixo dados completos

ax.plot(tabela_internados_datas['data_sint'],
        tabela_internados_datas['media_movel_internados'],
        color='dimgrey',
        alpha=.7, 
        linestyle='--')

ax.bar(tabela_internados_datas['data_sint'],
       tabela_internados_datas['internados_dia'],
       color='dimgrey', 
       alpha=.3)

#amarelo por cima até três semanas antes

ax.plot(tabela_incompleta_tres_semanas['data_sint'],
        tabela_incompleta_tres_semanas['media_movel_internados'],
        color='darkorange',
        alpha=1)

ax.bar(tabela_incompleta_tres_semanas['data_sint'],
       tabela_incompleta_tres_semanas['internados_dia'],
       color='darkorange', 
       alpha=.3)

ax.set_ylim(0, tabela_internados_datas['internados_dia'].max())
ax.set_xlim(left=tabela_internados_datas['data_sint'].min(), right=tabela_internados_datas['data_sint'].max())


ax.set_ylabel("Novas internações")

ax.set_xlabel("\nFonte: Sivep/OpenDataSUS. Análise: Cristiano Martins / @coronavirus_mg", color='grey', size=10)

ax.set_title("INTERNAÇÕES POR COVID-19 EM MINAS GERAIS\n(média móvel de sete dias, por data dos primeiros sintomas)")

ax.annotate('      '+INTERNADOS_MEDIA21, 
            xy = (pd.Timestamp(data_tres_semanas_atras), internados_media21-20),
            size = 16,
            weight ='bold')


ax.annotate('*Atraso\nmédio nas\nnotificações', 
            xy = (pd.Timestamp(internados_data_recente - datetime.timedelta(days=90)), tabela_incompleta_tres_semanas['media_movel_internados'].max()),
            size = 10, 
            horizontalalignment='center')


#ax.annotate('       '+OBITOS_VAR, 
#            xy = (pd.Timestamp(date.today()), obitos_media - 30),#-1300),
#            size = 13, 
#            color = cor_obitos)

ax.axhline(y = internados_media21, color = 'grey', linestyle = 'dotted')
ax.axhline(y = 0, color = 'grey', linestyle = '-')

#ax.axvline(x = data_tres_semanas_atras, color = 'grey', linestyle = 'dashed')
ax.axvspan(data_tres_semanas_atras, internados_data_recente, color='dimgrey', alpha=0.1)


#ax.text(0.5, -0.15, 'Fonte: SES-MG. Análise: Cristiano Martins / @coronavirus_mg', color='grey', size=11, horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)

fig.set_size_inches([16/2,9/2])

fig.savefig('grafico_media_internações.png', dpi = 300)

#### Gráfico das letalidades proporcionais

In [ ]:
pct_de_contaminados_total = [100-letalidade, 100-letalidade_internados, 100-letalidade_internados_UTI]
taxas_pct_letalidade = [letalidade, letalidade_internados, letalidade_internados_UTI]
categoria_letalidade = ['Geral', 'Internados', 'UTIs']

dados_grafico_letalidade = pd.DataFrame({'taxas_pct_letalidade': taxas_pct_letalidade, 
                                         'pct_de_contaminados_total': pct_de_contaminados_total}, 
                                        index=categoria_letalidade)

In [ ]:
#import matplotlib.pyplot as plt

plt.style.use('default')

fig, ax = plt.subplots()

ax.set_title('LETALIDADE DA COVID-19 EM MINAS GERAIS', pad=10, size=10, loc='center')

ax.barh(categoria_letalidade, 100, color='lightgrey')
ax.barh(categoria_letalidade, taxas_pct_letalidade, color='indianred')

ax.set_xlabel("Fonte: Sivep/OpenDataSUS e SES-MG  •  Análise: Cristiano Martins/@coronavirus_mg", labelpad=9, color= 'grey', size=9)
ax.set_xticks([0,25,50,75,100])
ax.set_xticklabels(['0%','25%','50%','75%','100%'], fontsize=9)
ax.xaxis.label.set_color('grey')
ax.tick_params(axis='x', colors='grey', size=5)
ax.tick_params(axis='y', length=0)

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(True)
ax.spines['bottom'].set_color('grey')

for i, v in enumerate(taxas_pct_letalidade):
    ax.text(v+1.5, i-.05, str(v.round(1))+'%', color='indianred', fontweight='bold')
    
fig.set_size_inches([16/2,9/2])

#ax.text(0.47, -0.3, 'Fonte: Sivep/OpenDataSUS e SES-MG. Análise: Cristiano Martins / @coronavirus_mg\n\n\n\n', color='grey', size=9, horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)

fig.savefig('grafico_letalidade.png', dpi = 300)

### Relatório em texto + publicar no Twitter

In [ ]:
templates01 = ["Minas Gerais registra OBITOS_24H novas mortes por #Covid19 e mais CASOS_24H casos confirmados da doença nDIA_DA_SEMANA.\n\nTotal no Estado chega a OBITOS_TOTAL óbitos, com CASOS_TOTAL pessoas infectadas desde o início da pandemia.\n\nFonte: @saudemg\n\n#coronavirus #mg #boletim", 
               "Mais OBITOS_24H mortes por #Covid19 foram confirmadas nDIA_DA_SEMANA em Minas. Com isso, Estado já registra OBITOS_TOTAL vítimas desde o início da pandemia.\n\nNúmero de infectados aumentou em CASOS_24H nas últimas 24h e chega a CASOS_TOTAL.\n\nFonte: @saudemg\n\n#coronavirus #mg #boletim", 
               "Número de vítimas da #Covid19 em Minas Gerais chega a OBITOS_TOTAL nDIA_DA_SEMANA, com OBITOS_24H novas mortes confirmadas desde ontem.\n\nForam registrados também mais CASOS_24H casos, elevando o total para CASOS_TOTAL infectados no Estado.\n\nFonte: @saudemg\n\n#coronavirus #mg #boletim", 
               "Minas Gerais confirmou mais OBITOS_24H mortes e CASOS_24H novos casos de #Covid19 nDIA_DA_SEMANA.\n\nAté agora, OBITOS_TOTAL pessoas morreram e CASOS_TOTAL foram infectadas pelo novo #coronavírus no Estado.\n\nFonte: @saudemg\n\n#mg #boletim", 
               "Mais OBITOS_24H mortes por #Covid19 e CASOS_24H novos casos da doença foram confirmados nDIA_DA_SEMANA em Minas Gerais.\n\nCom a atualização, Estado registra ao todo OBITOS_TOTAL vítimas e CASOS_TOTAL pessoas infectadas.\n\nFonte: @saudemg\n\n#coronavirus #mg #boletim", 
               "Minas Gerais registra OBITOS_24H novas mortes por #Covid19 nDIA_DA_SEMANA. Ao todo, doença já matou OBITOS_TOTAL mineiros.\n\nTambém foram confirmados CASOS_24H novos casos, e total de infectados chega a CASOS_TOTAL no Estado.\n\nFonte: @saudemg\n\n#coronavirus #mg #boletim", 
               "Nas últimas 24 horas, Minas Gerais confirmou mais OBITOS_24H óbitos e CASOS_24H habitantes diagnosticados com #Covid19.\n\nTotais no Estado chegam a OBITOS_TOTAL vítimas e CASOS_TOTAL casos nDIA_DA_SEMANA.\n\nFonte: @saudemg\n\n#coronavirus #mg #boletim", 
               "De ontem para hoje, foram registrados mais OBITOS_24H óbitos por #Covid19 e CASOS_24H novos diagnósticos da doença em Minas.\n\nNovo #coronavírus já ceifou OBITOS_TOTAL vidas e infectou CASOS_TOTAL residentes no Estado até DIA_DA_SEMANA.\n\nFonte: @saudemg\n\n#mg #boletim", 
               "Total de vidas perdidas para a #Covid19 em Minas chega a OBITOS_TOTAL, com mais OBITOS_24H mortes registradas nDIA_DA_SEMANA.\n\nTambém foram confirmados CASOS_24H novos casos, totalizando CASOS_TOTAL mineiros infectados pelo novo #coronavírus.\n\nFonte: @saudemg\n\n#mg #boletim", 
               "Mais OBITOS_24H óbitos e CASOS_24H novos casos de #Covid19 foram confirmados em Minas nDIA_DA_SEMANA.\n\nEstado já contabiliza ao todo OBITOS_TOTAL mortes e CASOS_TOTAL pessoas infectadas pelo novo #coronavírus desde o início da pandemia.\n\nFonte: @saudemg\n\n#mg #boletim"
              ]

In [ ]:
from random import randint

# selecionar aleatoriamente um índice entre 0 e 9 (lista tem 10 índices)
indice = randint(0, 9)
# selecionar o template no índice especificado
template_selecionado01 = templates01[indice]

In [ ]:
template_selecionado01 = str(template_selecionado01\
                           .replace('OBITOS_24H', OBITOS_24H)
                           .replace('DIA_DA_SEMANA', DIA_DA_SEMANA)
                           .replace('OBITOS_TOTAL', OBITOS_TOTAL)
                           .replace('CASOS_24H', CASOS_24H)
                           .replace('CASOS_TOTAL', CASOS_TOTAL))

print(template_selecionado01)
len(template_selecionado01) < 280

### Publicar no Twitter

In [ ]:
import twitter

#Inserir chaves credenciais da API do Twitter

api = twitter.Api(consumer_key=" ",
                  consumer_secret=" ",
                  access_token_key=" ",
                  access_token_secret=" ")

In [ ]:
#if tabela_obitos.index[-1] != pd.Timestamp(data_de_hoje) and tabela_confirmados.index[-1] != pd.Timestamp(data_de_hoje):
if boletim_atualizado == False:  
    mensagem_erro = "😕 Infelizmente, parece que a @SaudeMG não atualizou os microdados da #Covid19 em Minas Gerais n"+DIA_DA_SEMANA+"."+"\n\nAssim que possível, nosso bot fará a atualização do boletim.\n\n#coronavirus #mg #boletim" 
    tweet = api.PostUpdate(mensagem_erro)
else:
    tweet = api.PostUpdate(template_selecionado01)

#### Média móvel de mortes + gráfico de mortes

In [ ]:
template_media_obitos = 'A média móvel de óbitos por #Covid19 em Minas Gerais encontra-se na casa das OBITOS_MEDIA novas mortes por dia.\n\nA variação registrada nDIA_DA_SEMANA é de OBITOS_VAR na comparação com duas semanas atrás, ou seja, OBITOS_VAR_STATUS.\n\n#coronavirus #mg #boletim'

In [ ]:
template_media_obitos = str(template_media_obitos\
                           .replace('OBITOS_MEDIA', OBITOS_MEDIA)
                           .replace('OBITOS_VAR_STATUS', OBITOS_VAR_STATUS)
                           .replace('OBITOS_VAR', OBITOS_VAR)
                           .replace('DIA_DA_SEMANA', DIA_DA_SEMANA))

print(template_media_obitos)
len(template_media_obitos) < 280

In [ ]:
if tabela_obitos.index[-1] != pd.Timestamp(data_de_hoje) and tabela_confirmados.index[-1] != pd.Timestamp(data_de_hoje):
    print("Status não foi atualizado")
else:
    tweet = api.PostUpdate(template_media_obitos,'grafico_media_obitos.png', in_reply_to_status_id=tweet.id)

#### Média móvel de casos + gráfico de casos

In [ ]:
template_media_casos = 'A média móvel dos casos confirmados de #Covid19 em Minas Gerais está em CASOS_MEDIA novos diagnósticos por dia.\n\nA variação observada nDIA_DA_SEMANA é de CASOS_VAR em relação a duas semanas atrás, isto é, CASOS_VAR_STATUS.\n\n#coronavirus #mg #boletim'

In [ ]:
template_media_casos = str(template_media_casos\
                           .replace('CASOS_MEDIA', CASOS_MEDIA)
                           .replace('CASOS_VAR_STATUS', CASOS_VAR_STATUS)
                           .replace('CASOS_VAR', CASOS_VAR)
                           .replace('DIA_DA_SEMANA', DIA_DA_SEMANA))

print(template_media_casos)
len(template_media_casos) < 280

In [ ]:
if tabela_obitos.index[-1] != pd.Timestamp(data_de_hoje) and tabela_confirmados.index[-1] != pd.Timestamp(data_de_hoje): 
    print("Status não foi atualizado")
else:
    tweet = api.PostUpdate(template_media_casos,'grafico_media_casos.png', in_reply_to_status_id=tweet.id)

#### Recuperados, ativos e gráfico de casos ativos

In [ ]:
template_ativos = f'Dos {CASOS_TOTAL} mineiros infectados:\n\n- {RECUPERADOS_TOTAL} se recuperaram ou não desenvolveram a doença;\n- {OBITOS_TOTAL} vieram a óbito;\n- {ATIVOS_ATUAL} estão neste momento internados ou em observação.\n\n#coronavirus #covid19 #mg #boletim'

In [ ]:
print(template_ativos)
len(template_ativos) < 280

In [ ]:
if tabela_obitos.index[-1] != pd.Timestamp(data_de_hoje) and tabela_confirmados.index[-1] != pd.Timestamp(data_de_hoje): 
    print("Status não foi atualizado")
else:
    tweet = api.PostUpdate(template_ativos,'grafico_ativos.png', in_reply_to_status_id=tweet.id)

#### Letalidades

In [ ]:
template_letalidade = f'A letalidade (óbitos/casos) do #coronavirus em Minas Gerais é de {LETALIDADE}.\n\nEntre os pacientes que desenvolveram a #Covid19 e precisaram de internação, essa taxa sobe para {LETALIDADE_INTERNADOS}.\n\nDos que demandaram atendimento em UTIs, {LETALIDADE_INTERNADOS_UTI} não saíram com vida.\n\n#mg #boletim'

In [ ]:
print(template_letalidade)
len(template_letalidade) < 280

In [ ]:
if tabela_obitos.index[-1] != pd.Timestamp(data_de_hoje) and tabela_confirmados.index[-1] != pd.Timestamp(data_de_hoje): 
    print("Status não foi atualizado")
else:
    tweet = api.PostUpdate(template_letalidade, 'grafico_letalidade.png', in_reply_to_status_id=tweet.id)

## PARTE 2 - Dados da vacinação

### Baixar arquivo do Vacinômetro da SES-MG
### Esse arquivo tem duas peculiaridades:
### 1) normalmente, só é atualizado de segunda a sexta-feira;
### 2) os dados são registrados no dia anterior ao boletim. Exemplo: os dados publicados no boletim de 25/08/2021 são os dados inseridos até 24/08/2021, diferentemente do que acontece no arquivo dos casos e óbitos.
### OBS: Por essas duas razões, os dados do boletim de segunda-feira se referem aos registros da sexta-feira anterior.


In [ ]:
vacinas_atualizadas = False
vacinas_segue_para_analise = False
contador_vacinas = 1

while vacinas_segue_para_analise == False:
    
    #baixar o arquivo
    wget.download('http://sescloud.saude.mg.gov.br/index.php/s/ZEzzC8jFpobXGjM/download?path=%2FVACINAS&files=XLSX_Vacinas.xlsx', 
                  'vacinas.xlsx')
    
    #ler o arquivo
    vacinas = pd.ExcelFile('vacinas.xlsx')
    lista_abas = []
    for aba in vacinas.sheet_names:
        abas = vacinas.parse(aba)
        lista_abas.append(abas)
    df = pd.concat(lista_abas)
        
    #checar se a data está atualizada, lembrando que os dados são do dia anterior
    #(às segundas-feiras, dados costumam entram como sendo da sexta anterior)
    
    data_vacinas_ok = None
    
    if pd.Timestamp(data_de_hoje).day_name() == "Monday":
        if datetime.datetime.date(df['Data'].max()) == (datetime.date.today() - datetime.timedelta(days=1)): 
            data_vacinas_ok = True
        elif datetime.datetime.date(df['Data'].max()) == (datetime.date.today() - datetime.timedelta(days=3)): 
            data_vacinas_ok = True 
        else: 
            data_vacinas_ok = False
    else: 
        if datetime.datetime.date(df['Data'].max()) == (datetime.date.today() - datetime.timedelta(days=1)): 
            data_vacinas_ok = True 
        else: 
            data_vacinas_ok = False
            
    #Se estiver atualizado, quebra o while
    
    if data_vacinas_ok == True:
        print('\nArquivo das vacinas está atualizado. Pode seguir para a análise.\n')
        vacinas_atualizadas = True
        vacinas_segue_para_analise = True
    
    #Se não estiver atualizado, apaga o arquivo e refaz o processo 10 vezes dando um intervalo de 15 minutos entre as tentativas
        
    else: 
        if contador_vacinas < 10: 
            os.remove('vacinas.xlsx')
            print(f'\nTentativa {contador_vacinas} fracassada. Tentar novamente em 15 minutos.\n') 
            contador_vacinas += 1
            time.sleep(900)
    
    #Se não estiver atualizado e falhar as 10 vezes, avisa que não atualizou e segue.
    #Na hora de tuitar, se o arquivo do boletim não estiver atualizado, vai tuitar uma mensagem de erro.
        
        else: 
            print('\nO arquivo das vacinas não foi atualizado, mas a vida precisa seguir adiante\n')
            vacinas_atualizadas = False
            vacinas_segue_para_analise = True

In [ ]:
df.tail(5)

In [ ]:
tabela_vacinas = df.groupby(['Data','Dose'],as_index = False)['Doses Aplicadas'].sum().pivot(index='Data',columns='Dose',values='Doses Aplicadas').reset_index().rename_axis(None)

tabela_vacinas['DOSE UNICA_total'] = tabela_vacinas['DOSE UNICA'].cumsum()
tabela_vacinas['PRIMEIRA DOSE_total'] = tabela_vacinas['PRIMEIRA DOSE'].cumsum()
tabela_vacinas['SEGUNDA DOSE_total'] = tabela_vacinas['SEGUNDA DOSE'].cumsum()

tabela_vacinas['D2/DU'] = tabela_vacinas['DOSE UNICA'] + tabela_vacinas['SEGUNDA DOSE']
tabela_vacinas['D2/DU_total'] = tabela_vacinas['DOSE UNICA_total'] + tabela_vacinas['SEGUNDA DOSE_total']

tabela_vacinas = tabela_vacinas.drop(["DOSE UNICA", "SEGUNDA DOSE", "DOSE UNICA_total", "SEGUNDA DOSE_total"], axis=1)

tabela_vacinas['media_movel_d1'] = tabela_vacinas['PRIMEIRA DOSE'].rolling(7).mean().round(1)
tabela_vacinas['media_movel_d2du'] = tabela_vacinas['D2/DU'].rolling(7).mean().round(1)

tabela_vacinas = tabela_vacinas.rename(columns={'PRIMEIRA DOSE': 'd1_dia', 
                                                'PRIMEIRA DOSE_total': 'd1_total', 
                                                'D2/DU' : 'd2du_dia', 
                                                'D2/DU_total' : 'd2du_total'})

In [ ]:
tabela_vacinas.tail(10)

In [ ]:
d1 = tabela_vacinas.iloc[-1,2].astype(int)
d2du = tabela_vacinas.iloc[-1,4].astype(int)
media_d1 = tabela_vacinas.iloc[-1,5].astype(int)
media_d2du = tabela_vacinas.iloc[-1,6].astype(int)
print('d1:',d1)
print('d2du:',d2du)
print('media_d1:',media_d1)
print('media_d2du:',media_d2du)

In [ ]:
D1 = str('{0:,}'.format(d1).replace(',','.'))
D2DU = str('{0:,}'.format(d2du).replace(',','.'))
print('D1:',D1)
print('D2DU:',D2DU)

In [ ]:
pop_total = 21411923
pop_adulta = 16508000

In [ ]:
pop_total_imunizada = ((d2du*100)/pop_total).round(1)
print('pop_total_imunizada:', pop_total_imunizada)

pop_total_d1 = ((d1*100)/pop_total).round(1)
print('pop_total_d1:', pop_total_d1)

pop_total_semvacina = (100-pop_total_d1).round(1)
print('pop_total_semvacina:', pop_total_semvacina)

pop_total_parcial = (pop_total_d1 - pop_total_imunizada).round(1)
print('pop_total_parcial:', pop_total_parcial)

POP_TOTAL_IMUNIZADA = str('{0:,}'.format(pop_total_imunizada).replace('.',',')) + '%'
print('POP_TOTAL_IMUNIZADA:',POP_TOTAL_IMUNIZADA)

POP_TOTAL_D1 = str('{0:,}'.format(pop_total_d1).replace('.',',')) + '%'
print('POP_TOTAL_D1:',POP_TOTAL_D1)

POP_TOTAL_SEMVACINA = str('{0:,}'.format(pop_total_semvacina).replace('.',',')) + '%'
print('POP_TOTAL_SEMVACINA:',POP_TOTAL_SEMVACINA)

POP_TOTAL_PARCIAL = str('{0:,}'.format(pop_total_parcial).replace('.',',')) + '%'
print('POP_TOTAL_PARCIAL:',POP_TOTAL_PARCIAL)


In [ ]:
pop_adulta_imunizada = ((d2du*100)/pop_adulta).round(1)
print('pop_adulta_imunizada:', pop_adulta_imunizada)

pop_adulta_d1 = ((d1*100)/pop_adulta).round(1)
print('pop_adulta_d1:', pop_adulta_d1)

pop_adulta_semvacina = (100-pop_adulta_d1).round(1)
print('pop_adulta_semvacina:', pop_adulta_semvacina)

pop_adulta_parcial = (pop_adulta_d1 - pop_adulta_imunizada).round(1)
print('pop_adulta_parcial:', pop_adulta_parcial)

POP_ADULTA_IMUNIZADA = str('{0:,}'.format(pop_adulta_imunizada).replace('.',',')) + '%'
print('POP_ADULTA_IMUNIZADA:',POP_ADULTA_IMUNIZADA)

POP_ADULTA_D1 = str('{0:,}'.format(pop_adulta_d1).replace('.',',')) + '%'
print('POP_ADULTA_D1:',POP_ADULTA_D1)

POP_ADULTA_SEMVACINA = str('{0:,}'.format(pop_adulta_semvacina).replace('.',',')) + '%'
print('POP_ADULTA_SEMVACINA:',POP_ADULTA_SEMVACINA)

POP_ADULTA_PARCIAL = str('{0:,}'.format(pop_adulta_parcial).replace('.',',')) + '%'
print('POP_ADULTA_PARCIAL:',POP_ADULTA_PARCIAL)


#### Gráfico da cobertura vacinal

In [ ]:
categoria_vacina = ['Pop. Total  ', 'Pop. Adulta  ']
camada1 = [pop_total_imunizada, pop_adulta_imunizada]
camada2 = [pop_total_d1, pop_adulta_d1]

In [ ]:
plt.style.use('default')

fig, ax = plt.subplots()

ax.set_title('COBERTURA VACINAL CONTRA A COVID-19 EM MINAS GERAIS', pad=10, size=10, loc='center')

ax.barh(categoria_vacina, 100, color='lightgrey', alpha=0.8)
ax.barh(categoria_vacina, camada2, color='mediumaquamarine', alpha=0.8)
ax.barh(categoria_vacina, camada1, color='seagreen', alpha=0.8)

ax.set_xlabel('Fonte: SES-MG/Prefeituras e IBGE  •  Análise: Cristiano Martins / @coronavirus_mg', labelpad=10, color= 'grey', size=8)
ax.set_xticks([0,20,40,60,80,100])
ax.set_xticklabels(['0%','20%','40%','60%','80%','100%'], fontsize=9)
ax.xaxis.label.set_color('grey')
ax.tick_params(axis='x', colors='grey', size=5)
ax.tick_params(axis='y', length=0)

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(True)
ax.spines['bottom'].set_color('grey')

for i, v in enumerate(camada1): 
    ax.text(v-12, i-.03, str(v.round(1))+'%', color='white', fontweight='bold')
    
for i, v in enumerate(camada2): 
    ax.text(v-12, i-.03, str(v.round(1))+'%', color='white', fontweight='bold')    

fig.set_size_inches([16/2,9/2])

#ax.text(0.47, -0.2, 'Fonte: Sivep/OpenDataSUS e SES-MG. Análise: Cristiano Martins / @coronavirus_mg', color='grey', size=9, horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)


fig.savefig('grafico_vacina.png', 
            dpi = 300)

In [ ]:
template_cobertura_vacina = f'Até agora, a vacinação contra a #Covid19 em Minas alcançou {D1} pessoas com a primeira dose e {D2DU} com a segunda/única.\n\nIsso representa {POP_TOTAL_IMUNIZADA} da população totalmente imunizada no Estado.\n\n#coronavirus #mg #boletim'

In [ ]:
print(template_cobertura_vacina)
len(template_cobertura_vacina) < 280

In [ ]:
vacinas_atualizadas

In [ ]:
mensagem_erro_vacinas = "😕 Infelizmente, parece que a @SaudeMG não atualizou os microdados da vacinação contra a Covid-19 em Minas Gerais n"+DIA_DA_SEMANA+"."+"\n\nAssim que possível, faremos a atualização do nosso relatório.\n\n#coronavirus #mg #boletim"
print(mensagem_erro_vacinas)

In [ ]:
#if tabela_obitos.index[-1] != pd.Timestamp(data_de_hoje) and tabela_confirmados.index[-1] != pd.Timestamp(data_de_hoje):
if vacinas_atualizadas == False:
    tweet = api.PostUpdate(mensagem_erro_vacinas, in_reply_to_status_id = tweet.id)
else: 
    tweet = api.PostUpdate(template_cobertura_vacina,'grafico_vacina.png', in_reply_to_status_id = tweet.id)

#### Ao final de todo o processo, renomear os arquixos e guardá-los em uma pasta de arquivo

In [ ]:
import os
import glob
import re
import shutil

In [ ]:
lista_arquivos = glob.glob('*.[px]*')
lista_arquivos

In [ ]:
for nome_arquivo in lista_arquivos: 
    nome_novo = f'{str(date.today())}'+'_'+nome_arquivo
    os.rename(nome_arquivo, nome_novo)
    
print('Os arquivos foram renomeados com sucesso')

In [ ]:
lista_arquivos = glob.glob('*.[px]*')
lista_arquivos

In [ ]:
for nome_arquivo in lista_arquivos: 
    shutil.move(nome_arquivo, "arquivo")
    
print('Os arquivos transferidos com sucesso')